## 📦 Instalação das Dependências

Execute o bloco abaixo para instalar todas as bibliotecas necessárias para o pipeline:

```python
# Bibliotecas principais para o pipeline ETL
!pip install pandas openpyxl pdfplumber beautifulsoup4 requests selenium webdriver-manager
```

### 🔎 Explicação rápida dos pacotes:

- `pandas`: manipulação de dados tabulares.
- `openpyxl`: leitura e escrita de arquivos `.xlsx`.
- `pdfplumber`: extração de tabelas de arquivos PDF.
- `beautifulsoup4`: parser HTML (usado para extrair links do site da ANVISA).
- `requests`: download de arquivos.
- `selenium`: controle do navegador para lidar com conteúdo dinâmico.
- `webdriver-manager`: instala e gerencia o ChromeDriver automaticamente.

In [ ]:
# Bibliotecas principais para o pipeline ETL
!pip install pandas openpyxl pdfplumber beautifulsoup4 requests selenium webdriver-manager

# 🗂️ Índice do Pipeline ETL - Roubos, Furtos e Extravios

1. [Etapa 1 - Coleta de Dados: Roubos, Furtos e Extravios no site da ANVISA](#etapa-1---coleta-de-dados-roubos-furtos-e-extravios-no-site-da-anvisa)
2. [Etapa 2 - Extração de Tabelas dos PDFs com `pdfplumber`](#etapa-2---extração-de-tabelas-dos-pdfs-com-pdfplumber)
3. [Etapa 3 - Tratamento e Padronização dos Arquivos Excel](#etapa-3---tratamento-e-padronização-dos-arquivos-excel)
4. [Etapa 4 - Consolidação Final dos Dados](#etapa-4---consolidação-final-dos-dados)

## 🔍 Etapa 1 - Coleta de Dados: Roubos, Furtos e Extravios no site da ANVISA

Nesta etapa, realizamos a **extração automatizada de arquivos** contendo registros de roubos, furtos e extravios de produtos sujeitos à vigilância sanitária, diretamente do portal da ANVISA:

🔗 **Fonte oficial:** [Portal ANVISA - Fiscalização e Monitoramento](https://www.gov.br/anvisa/pt-br/assuntos/fiscalizacao-e-monitoramento/roubos-furtos-e-extravios)

---

### 📌 Objetivo

Automatizar o processo de **extração e download** dos arquivos publicados no site da ANVISA, organizando-os por tipo de produto e ano. Os dados coletados são essenciais para alimentar a próxima etapa do processo ETL.

---

### 🧰 Ferramentas Utilizadas

- **Selenium**: renderiza o conteúdo dinâmico da página (JavaScript).
- **BeautifulSoup**: realiza o parse do HTML e extrai os links relevantes.
- **Requests**: efetua o download dos arquivos encontrados.
- **Regex (re)**: identifica o tipo do produto e o ano a partir dos nomes dos links.

---

### 📁 Organização dos dados baixados

Os arquivos são organizados da seguinte forma:

```
dados/
├── 2014/
│   ├── medicamentos_2014.xlsx
│   └── cosmeticos_2014.pdf
├── 2015/
│   └── produtos_saude_2015.xlsx
├── 2018/
│   └── medicamentos_2018.xlsx  ← adicionados manualmente
...
```

---

### 🧠 Lógica do Script

1. **Acessa a URL base** com Selenium e espera o carregamento da página.
2. **Coleta todos os links** do HTML com BeautifulSoup.
3. **Filtra os links** que contêm:
   - Um ano (ex: `2019`)
   - Um dos tipos de produto mapeados:
     - medicamentos
     - produtos para saúde
     - cosméticos
     - alimentos
     - carga
4. **Determina o tipo de produto e extensão do arquivo** (.pdf, .xlsx etc.).
5. **Cria pastas por ano** (se necessário) e salva o arquivo no caminho correspondente.
6. **Registra o sucesso ou falha** de cada download com mensagens de log.

---

### ✅ Resultado Esperado

Ao final da execução, teremos um conjunto de arquivos .pdf ou .xlsx organizados por ano e tipo de produto, prontos para o processamento na próxima etapa do pipeline ETL (extração de dados tabulares, limpeza, padronização e consolidação).

---

### 🛡️ Observações

- O script trata exceções e links inválidos.
- URLs com extensão ambígua são tratadas como PDF por padrão.
- O uso do `webdriver-manager` facilita a instalação do ChromeDriver automaticamente.

> ⚠️ **Importante:** Os arquivos referentes ao **ano de 2018** não seguem o mesmo padrão de nomeação ou estrutura de links automatizáveis. Portanto, eles devem ser **baixados manualmente do site**, **renomeados conforme os demais arquivos** (ex: `medicamentos_2018.xlsx`) e salvos na pasta:
>
> ```
> dados/2018/
> ```

---

In [ ]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# === CONFIGURAÇÃO ===
URL_BASE = "https://www.gov.br/anvisa/pt-br/assuntos/fiscalizacao-e-monitoramento/roubos-furtos-e-extravios"
PASTA_DADOS = "dados"
os.makedirs(PASTA_DADOS, exist_ok=True)

TIPOS_PRODUTO = {
    "medicamentos": "medicamentos",
    "produtos para a saúde": "produtos_saude",
    "produtos para saúde": "produtos_saude",
    "cosméticos": "cosmeticos",
    "alimentos": "alimentos",
    "carga": "carga"  # Adicionado para capturar planilhas como "Roubo de Carga"
}

# === ABRIR PÁGINA COM SELENIUM ===
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(URL_BASE)
time.sleep(3)  # Espera o conteúdo carregar

html = driver.page_source
driver.quit()

# === PARSE COM BEAUTIFULSOUP ===
sopa = BeautifulSoup(html, "html.parser")
links = sopa.find_all("a", href=True)

# === PROCESSAR LINKS ===
for link in links:
    texto = link.get_text(strip=True).lower()
    href = link["href"]

    # Detectar ano
    ano_match = re.search(r"(\d{4})", texto)
    if not ano_match:
        continue
    ano = ano_match.group(1)

    # Detectar tipo de produto
    tipo_match = next((TIPOS_PRODUTO[tipo] for tipo in TIPOS_PRODUTO if tipo in texto), None)
    if not tipo_match:
        continue

    # Detectar extensão real
    extensao = href.split(".")[-1].split("?")[0].lower()
    if len(extensao) > 5 or "/" in extensao:
        extensao = "pdf"  # Força PDF apenas se extensão for estranha

    # Nome do arquivo
    nome_arquivo = f"{tipo_match}_{ano}.{extensao}"
    pasta_ano = os.path.join(PASTA_DADOS, ano)
    os.makedirs(pasta_ano, exist_ok=True)
    caminho_arquivo = os.path.join(pasta_ano, nome_arquivo)

    # Construir URL completa
    url_arquivo = urljoin(URL_BASE, href)

    # Baixar o arquivo
    try:
        r = requests.get(url_arquivo, allow_redirects=True)
        if r.status_code == 200:
            with open(caminho_arquivo, "wb") as f:
                f.write(r.content)
            print(f"✅ Arquivo salvo: {caminho_arquivo}")
        else:
            print(f"⚠️ Erro HTTP {r.status_code}: {url_arquivo}")
    except Exception as e:
        print(f"❌ Erro ao baixar {url_arquivo}: {e}")

## 📄 Etapa 2 - Extração de Tabelas dos PDFs

Após a coleta automatizada dos arquivos PDF no site da ANVISA, esta etapa do pipeline tem como objetivo **extrair as tabelas contidas nos arquivos PDF**, tratando e estruturando os dados em formato tabular (Excel `.xlsx`).

Dois métodos de extração são utilizados, dependendo da estrutura do PDF:

- **pdfplumber**: ideal para tabelas simples com separações baseadas em espaçamento.
- **Camelot (modo `lattice`) + PyPDF2**: necessário para PDFs com tabelas estruturadas por linhas desenhadas ou com formatação irregular.

---

### 🧰 Ferramentas Utilizadas

- **pdfplumber**: biblioteca para extração de tabelas de PDFs com estrutura tabular simples.
- **Camelot**: utilizada para detectar tabelas baseadas em linhas desenhadas (modo `lattice`).
- **PyPDF2**: leitura do número de páginas dos PDFs.
- **pandas**: consolidação e tratamento dos dados tabulados.
- **Regex**: remoção de caracteres ilegais para compatibilidade com Excel.
- **gc**: liberação manual da memória ao fim do processamento de cada arquivo.

---

### 🧠 Lógica dos Scripts

#### 📘 Etapa 2.1 – Extração com `pdfplumber`

1. **Varre recursivamente a pasta `dados/`**, onde os arquivos PDF foram salvos.
2. Para cada arquivo:
   - Identifica o **tipo do produto** (ex: `medicamentos`) e o **ano** a partir do nome do arquivo (ex: `medicamentos_2018.pdf`).
   - Realiza a **extração das tabelas** contidas em todas as páginas com `pdfplumber`.
   - Cria um DataFrame consolidado contendo todas as tabelas extraídas.
   - Adiciona uma nova coluna chamada `"Tipo do Produto"` com o valor legível (`medicamentos`, `cosméticos`, etc).
3. **Remove caracteres ilegais** com regex para garantir compatibilidade com o Excel.
4. Salva o resultado como `.xlsx` na pasta `dataframe/ano`.

#### 📙 Etapa 2.2 – Extração com `Camelot` + `PyPDF2`

1. Reprocessa todos os arquivos `.pdf` na pasta `dados/`, página por página.
2. Para cada PDF:
   - Usa `PyPDF2` para identificar o número de páginas.
   - Tenta extrair tabelas usando `Camelot` com o modo `"lattice"`.
   - Aplica funções de limpeza:
     - Define corretamente o cabeçalho da tabela.
     - Remove repetições de cabeçalhos entre páginas.
     - Corrige quebras de índice, caso número e descrição estejam juntos.
   - Adiciona a coluna `"Tipo do Produto"` com base no nome do arquivo.
   - Remove caracteres ilegais com regex.
3. Salva o resultado final como `.xlsx` na pasta `dataframe/ano/`.
4. Libera a memória ao final do processamento de cada PDF.

---

### 🧼 Tratamento de Caracteres Inválidos

Certos PDFs podem conter caracteres invisíveis ou inválidos (ex: `\x0B`, `\x0C`), que causam erro ao salvar em Excel. Ambos os scripts utilizam uma expressão regular para limpar todas as colunas do tipo texto.

```python
ILLEGAL_EXCEL_CHARS = re.compile(r'[\x00-\x08\x0B\x0C\x0E-\x1F]')
```

---

### 📁 Organização da Saída

A saída da extração (de ambos os métodos) é salva em arquivos `.xlsx`, um por PDF, na pasta:

```
dataframe/ano/
├── medicamentos_2011.xlsx
├── cosmeticos_2013.xlsx
└── alimentos_2018.xlsx
```

Cada planilha terá uma coluna adicional: `Tipo do Produto`.

---

### ✅ Resultado Esperado

Arquivos Excel com os dados extraídos e estruturados de forma padronizada, prontos para serem tratados e consolidados na próxima etapa do ETL.

---

### 🚨 Observações

- Os dois scripts processam somente arquivos `.pdf`.
- PDFs sem tabelas detectáveis serão ignorados.
- Tipos de produto são mapeados automaticamente a partir do nome do arquivo, com base no dicionário:

```python
TIPOS_HUMAN = {
    "medicamentos": "medicamentos",
    "produtos_saude": "produtos para saúde",
    "cosmeticos": "cosméticos",
    "alimentos": "alimentos",
    "carga": "carga"
}
```

> ⚠️ **Importante:** Os arquivos `produtos_saude_2014.pdf` e `cosmeticos_2015.pdf` foram salvos com formatação que impede a identificação correta das colunas pelo `pdfplumber`. 
> 
> Portanto, após a conversão automática, será necessário **abrir manualmente os arquivos `.xlsx` gerados**, e:
>
> - Copiar a **estrutura de colunas de um arquivo semelhante** (ex: `medicamentos_2014.xlsx`, `medicamentos_2015.xlsx`)
> - **Colar como cabeçalho** nos arquivos problemáticos.
>
> Isso garante que a padronização e consolidação nas etapas seguintes funcione corretamente.

---

### 🔁 Ordem Recomendada de Execução

1. **Execute primeiro o script com `pdfplumber`** para processar os arquivos com estrutura simples.
2. **Depois, execute o script com `Camelot`**, que tentará reprocessar os arquivos restantes, corrigindo falhas do passo anterior.

> Essa sequência garante a máxima recuperação dos dados e evita sobrescritas desnecessárias.

In [ ]:
import os
import re
import pandas as pd
import pdfplumber

# === CONFIGURAÇÕES ===
DATA_DIR = "dados"
OUTPUT_DIR = "dataframe/ano"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Lista de arquivos desejados (sem extensão)
arquivos_desejados = [
    "produtos_saude_2014",
    "cosmeticos_2015",
    "produtos_saude_2017"
]

TIPOS_HUMAN = {
    "medicamentos": "Medicamento",
    "produtos_saude": "Produtos para Saude",
    "cosmeticos": "Cosmético",
    "alimentos": "Alimento"
}

ILLEGAL_EXCEL_CHARS = re.compile(r'[\x00-\x08\x0B\x0C\x0E-\x1F]')

def limpar_caracteres_invalidos(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        try:
            if pd.api.types.is_object_dtype(df[col]):
                df[col] = df[col].astype(str).str.replace(ILLEGAL_EXCEL_CHARS, "", regex=True)
        except Exception as e:
            print(f"⚠️ Erro ao limpar coluna '{col}': {e}")
    return df

# === PROCESSAMENTO DOS PDFs ===
for root, dirs, files in os.walk(DATA_DIR):
    for filename in files:
        slug = os.path.splitext(filename)[0]  # nome sem extensão
        if filename.lower().endswith(".pdf") and slug in arquivos_desejados:
            filepath = os.path.join(root, filename)
            parts = slug.split("_")

            year = parts[-1]
            tipo_slug = "_".join(parts[:-1])
            tipo_human = TIPOS_HUMAN.get(tipo_slug, tipo_slug)

            dfs = []
            with pdfplumber.open(filepath) as pdf:
                for page in pdf.pages:
                    table = page.extract_table()
                    if table:
                        df = pd.DataFrame(table[1:], columns=table[0])
                        dfs.append(df)

            df_all = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
            df_all["Tipo do Produto"] = tipo_human
            df_all = limpar_caracteres_invalidos(df_all)

            output_filename = f"{slug}.xlsx"
            output_path = os.path.join(OUTPUT_DIR, output_filename)
            df_all.to_excel(output_path, index=False)

            print(f"✔ Gerado: {output_path}")

In [ ]:
import os
import re
import gc
import pandas as pd
import camelot
from PyPDF2 import PdfReader

# === CONFIGURAÇÕES ===
DATA_DIR = "dados"
OUTPUT_DIR = "dataframe/ano"
os.makedirs(OUTPUT_DIR, exist_ok=True)

TIPOS_HUMAN = {
    "medicamentos": "Medicamento",
    "produtos_saude": "Produtos para Saude",
    "cosmeticos": "Cosmético",
    "alimentos": "Alimento"
}

ILLEGAL_EXCEL_CHARS = re.compile(r'[\x00-\x08\x0B\x0C\x0E-\x1F]')

def limpar_caracteres_invalidos(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        try:
            if pd.api.types.is_object_dtype(df[col]):
                df[col] = df[col].astype(str).str.replace(ILLEGAL_EXCEL_CHARS, "", regex=True)
        except Exception as e:
            print(f"⚠️ Erro ao limpar coluna '{col}': {e}")
    return df

def corrigir_indice_duplicado(df: pd.DataFrame) -> pd.DataFrame:
    if df.shape[1] < 2:
        return df

    col0 = df.columns[0]
    col1 = df.columns[1]
    padrao = re.compile(r"^(\d{1,4})\s+(.*)")

    for i in df.index:
        val0 = str(df.at[i, col0]).strip() if pd.notna(df.at[i, col0]) else ""
        val1 = str(df.at[i, col1]).strip() if pd.notna(df.at[i, col1]) else ""

        match0 = padrao.match(val0)
        match1 = padrao.match(val1)

        if val0 == "" and match1:
            numero, texto = match1.groups()
            df.at[i, col0] = numero
            df.at[i, col1] = texto
        elif match1 and match1.group(1) == val0:
            df.at[i, col1] = match1.group(2)
        elif match0 and val1 == "":
            numero, texto = match0.groups()
            df.at[i, col0] = numero
            df.at[i, col1] = texto

    return df

def padronizar_tabela(tabela: pd.DataFrame) -> pd.DataFrame:
    tabela.columns = tabela.iloc[0]
    tabela = tabela.drop(index=0).reset_index(drop=True)
    tabela = tabela[~tabela.eq(tabela.columns).all(axis=1)]
    tabela = corrigir_indice_duplicado(tabela)
    return tabela

# === PROCESSAMENTO DOS PDFs ===
for root, dirs, files in os.walk(DATA_DIR):
    for filename in files:
        if filename.lower().endswith(".pdf"):
            filepath = os.path.join(root, filename)
            slug = os.path.splitext(filename)[0]
            parts = slug.split("_")

            year = parts[-1]
            tipo_slug = "_".join(parts[:-1])
            tipo_human = TIPOS_HUMAN.get(tipo_slug, tipo_slug)

            try:
                reader = PdfReader(filepath)
                num_pages = len(reader.pages)
                dfs = []

                for page_num in range(1, num_pages + 1):
                    try:
                        tables = camelot.read_pdf(filepath, pages=str(page_num), flavor="lattice")
                        for table in tables:
                            df = padronizar_tabela(table.df)
                            dfs.append(df)
                    except Exception as e_page:
                        print(f"⚠️ Erro na página {page_num} de {filename}: {e_page}")

                df_all = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

                if not df_all.empty:
                    df_all["Tipo do Produto"] = tipo_human
                    df_all = limpar_caracteres_invalidos(df_all)

                    output_filename = f"{slug}.xlsx"
                    output_path = os.path.join(OUTPUT_DIR, output_filename)
                    df_all.to_excel(output_path, index=False)
                    print(f"✔ Gerado: {output_path}")
                else:
                    print(f"⚠️ Nenhuma tabela encontrada em {filename}")

                # === Liberação de memória ===
                del reader, dfs, tables, df_all
                gc.collect()

            except Exception as e:
                print(f"❌ Erro ao processar {filename}: {e}")

## 🧹 Etapa 3 - Tratamento e Padronização dos Arquivos Excel

Após a extração dos dados tabulares dos PDFs, esta etapa tem como objetivo **limpar, padronizar e consolidar os arquivos `.xlsx`** gerados anteriormente, garantindo consistência nas colunas e estrutura dos dados ao longo dos anos.

---

### 🧠 Lógica do Script

1. **Varre os arquivos da pasta `dataframe/ano/`**, agrupando os arquivos por ano com base no nome do arquivo.
2. Para cada planilha:
   - Remove colunas irrelevantes como `"Ordem"` e `"Unnamed: 0"`.
   - **Normaliza os nomes das colunas** (remove asteriscos, espaços duplicados, etc).
   - **Renomeia colunas** de acordo com um dicionário (`col_map`) para garantir padronização (ex: `"Produto"` → `"PRODUTO"`).
   - **Remove colunas duplicadas**, caso existam.
   - Garante a existência da coluna `"UF"`; se estiver ausente, ela é criada.
   - Quando a cidade e o estado estiverem juntos na coluna `"CIDADE"` (ex: `"ATIBAIA - SP"`), o script separa os valores em duas colunas: `"CIDADE"` e `"UF"`.
   - Converte a sigla da UF para o formato `"Nome do Estado (UF)"` (ex: `"SP"` → `"São Paulo (SP)"`).
3. **Reorganiza as colunas** na ordem definida para facilitar análises posteriores.
4. **Consolida todos os arquivos de um mesmo ano** em um único DataFrame e exporta para a pasta `dataframe/tratado/`.

---

### 🧾 Mapeamento de Colunas

O script utiliza o seguinte dicionário de padronização para uniformizar os nomes das variáveis, independentemente das variações que aparecem nos arquivos:

```python
col_map = {
    "Medicamento": "PRODUTO",
    "Produto Saúde": "PRODUTO",
    "Produto": "PRODUTO",
    "Empresa": "EMPRESA",
    "Nº lote": "N_LOTE",
    "Nº Lote": "N_LOTE",
    "Nº de Série/ Lote": "N_LOTE",
    "Quantidade Roubada/Furtada/Extraviada": "QUANTIDADE_ROUBADA_FURTADA_EXTRAVIADA",
    "Quantidade Roubada/Furtada": "QUANTIDADE_ROUBADA_FURTADA_EXTRAVIADA",
    "Quantidade Roubada/ Furtada/Extraviada": "QUANTIDADE_ROUBADA_FURTADA_EXTRAVIADA",
    "Quantidade Roubada/Furtada/ Extraviada": "QUANTIDADE_ROUBADA_FURTADA_EXTRAVIADA",
    "Nº Nota Fiscal": "N_NOTA_FISCAL",
    "Nº Boletim de Ocorrência": "N_BOLETIM_DE_OCORRENCIA",
    "Data do roubo ou extravio": "DATA_DO_ROUBO_OU_EXTRAVIO",
    "Local do roubo ou extravio": "CIDADE",
    "Cidade": "CIDADE",
    "UF": "UF",
    "Tipo do Produto": "TIPO_DE_PRODUTO"
}
```

---

### 📁 Organização da Saída

Os arquivos tratados e consolidados são salvos por ano na pasta:

```
dataframe/tratado/
├── 2011_tratado.xlsx
├── 2012_tratado.xlsx
└── ...
```

---

### ✅ Resultado Esperado

Ao final da execução, os dados estarão estruturados, consistentes e prontos para as próximas etapas de análise, integração ou visualização, com colunas padronizadas e sem duplicações ou inconsistências regionais.

---


In [ ]:
import os
import pandas as pd
import re

# === CONFIGURAÇÕES ===
INPUT_FOLDER = "dataframe/ano"
OUTPUT_FOLDER = "dataframe/tratado"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Mapeamento de colunas após limpeza
col_map = {
    "Medicamento": "PRODUTO",
    "Produto Saúde": "PRODUTO",
    "Produto": "PRODUTO",
    "Empresa": "EMPRESA",
    "Nº lote": "N_LOTE",
    "Nº Lote": "N_LOTE",
    "Nº de Série/ Lote": "N_LOTE",
    "Quantidade Roubada/Furtada/Extraviada": "QUANTIDADE_ROUBADA_FURTADA_EXTRAVIADA",
    "Quantidade Roubada/Furtada": "QUANTIDADE_ROUBADA_FURTADA_EXTRAVIADA",
    "Quantidade Roubada/ Furtada/Extraviada": "QUANTIDADE_ROUBADA_FURTADA_EXTRAVIADA",
    "Quantidade Roubada/Furtada/ Extraviada": "QUANTIDADE_ROUBADA_FURTADA_EXTRAVIADA",
    "Nº Nota Fiscal": "N_NOTA_FISCAL",
    "Nº Boletim de Ocorrência": "N_BOLETIM_DE_OCORRENCIA",
    "Data do roubo ou extravio": "DATA_DO_ROUBO_OU_EXTRAVIO",
    "Local do roubo ou extravio": "CIDADE",
    "Cidade": "CIDADE",
    "UF": "UF",
    "Tipo do Produto": "TIPO_DE_PRODUTO"
}

ordem_final = [
    "TIPO_DE_PRODUTO", "PRODUTO", "EMPRESA", "N_LOTE",
    "QUANTIDADE_ROUBADA_FURTADA_EXTRAVIADA",
    "N_BOLETIM_DE_OCORRENCIA", "N_NOTA_FISCAL",
    "DATA_DO_ROUBO_OU_EXTRAVIO", "CIDADE", "UF"
]

# Mapa UF → Nome do estado
UF_ESTADO = {
    "AC": "Acre", "AL": "Alagoas", "AP": "Amapá", "AM": "Amazonas", "BA": "Bahia",
    "CE": "Ceará", "DF": "Distrito Federal", "ES": "Espírito Santo", "GO": "Goiás",
    "MA": "Maranhão", "MT": "Mato Grosso", "MS": "Mato Grosso do Sul", "MG": "Minas Gerais",
    "PA": "Pará", "PB": "Paraíba", "PR": "Paraná", "PE": "Pernambuco", "PI": "Piauí",
    "RJ": "Rio de Janeiro", "RN": "Rio Grande do Norte", "RS": "Rio Grande do Sul",
    "RO": "Rondônia", "RR": "Roraima", "SC": "Santa Catarina", "SP": "São Paulo",
    "SE": "Sergipe", "TO": "Tocantins"
}

# Função para separar cidade e UF quando estiverem juntas
def separar_cidade_uf(row):
    cidade = row["CIDADE"]
    uf = row["UF"]
    if pd.isna(cidade):
        return cidade, uf
    match = re.match(r"(.+?)[\s,-]+([A-Z]{2})$", str(cidade).strip(), re.IGNORECASE)
    if match:
        cidade_extraida = match.group(1).strip().title()
        uf_extraida = match.group(2).upper()
        return cidade_extraida, uf_extraida
    return cidade, uf

# Função para formatar UF como "Estado (UF)"
def formatar_estado_completo(sigla):
    if pd.isna(sigla):
        return None
    sigla = str(sigla).strip().upper()
    if sigla in UF_ESTADO:
        return f"{UF_ESTADO[sigla]} ({sigla})"
    return sigla  # mantém valor original se não reconhecido

# Agrupar arquivos por ano
arquivos_por_ano = {}
for arquivo in os.listdir(INPUT_FOLDER):
    if arquivo.endswith(".xlsx"):
        ano_match = re.search(r"(\d{4})", arquivo)
        if ano_match:
            ano = ano_match.group(1)
            arquivos_por_ano.setdefault(ano, []).append(arquivo)

# Processar cada ano
for ano, arquivos in sorted(arquivos_por_ano.items()):
    print(f"\n📆 Consolidando ano {ano}...")
    dfs = []

    for arquivo in arquivos:
        caminho = os.path.join(INPUT_FOLDER, arquivo)
        print(f"   📄 Processando: {arquivo}")
        try:
            df = pd.read_excel(caminho)

            # Limpeza de colunas
            df.columns = (
                df.columns
                .str.replace(r"\s+", " ", regex=True)
                .str.replace("*", "", regex=False)
                .str.strip()
            )

            print(f"   🔍 Colunas normalizadas: {list(df.columns)}")

            # Remover colunas ignoradas
            df = df[[col for col in df.columns if col not in ["Ordem", "Unnamed: 0"]]]

            # Renomear colunas
            df = df.rename(columns={k: v for k, v in col_map.items() if k in df.columns})

            # Remover colunas duplicadas
            duplicadas = df.columns[df.columns.duplicated()].tolist()
            if duplicadas:
                print(f"   ⚠️ Colunas duplicadas removidas: {duplicadas}")
                df = df.loc[:, ~df.columns.duplicated()]

            # Garantir existência da coluna UF
            if "UF" not in df.columns:
                df["UF"] = None

            # Extrair cidade e UF se necessário
            if "CIDADE" in df.columns:
                df[["CIDADE", "UF"]] = df.apply(separar_cidade_uf, axis=1, result_type="expand")

            # Formatar UF para "Estado (UF)"
            df["UF"] = df["UF"].apply(formatar_estado_completo)

            # Reorganizar colunas
            colunas_existentes = [col for col in ordem_final if col in df.columns]
            df = df[colunas_existentes]

            print(f"   ✅ Colunas finais: {list(df.columns)}")
            dfs.append(df)

        except Exception as e:
            print(f"   ❌ Erro ao processar {arquivo}: {e}")

    # Consolidar e salvar
    if dfs:
        df_ano = pd.concat(dfs, ignore_index=True)
        output_path = os.path.join(OUTPUT_FOLDER, f"{ano}_tratado.xlsx")
        df_ano.to_excel(output_path, index=False)
        print(f"📁 Arquivo salvo: {output_path}")
    else:
        print(f"⚠️ Nenhum dado válido encontrado para {ano}.")

## 🧩 Etapa 4 - Consolidação Final dos Dados

Nesta etapa final, todos os arquivos `.xlsx` previamente tratados e estruturados são **lidos, unificados e padronizados** em um único DataFrame, resultando na base final consolidada.

---

### 🧠 Lógica do Script

1. **Lê todos os arquivos da pasta `dataframe/tratado/`**:
   - Todos os arquivos `.xlsx` são carregados e armazenados em uma lista de DataFrames.
2. **Lê também arquivos Excel adicionais** que estejam presentes nas subpastas da pasta `dados/`, **exceto a pasta `dados/2023/`**, que é explicitamente ignorada.
3. **Concatena todos os DataFrames** coletados, removendo possíveis duplicatas.
4. **Padroniza os valores da coluna `TIPO_DE_PRODUTO`** para corrigir variações nos nomes (ex: `"cosméticos"` → `"Cosmético"`).
5. **Salva o DataFrame final** no arquivo `consolidado_final.xlsx`, na pasta `dataframe/consolidado`.

---

### 🎯 Padronização de `TIPO_DE_PRODUTO`

A coluna `TIPO_DE_PRODUTO` é harmonizada com o seguinte mapeamento, consolidando grafias diferentes em categorias padronizadas:

```python
mapeamento = {
    "alimentos": "Alimento",
    "alimento": "Alimento",
    "cosméticos": "Cosmético",
    "cosmeticos": "Cosmético",
    "medicamentos": "Medicamento",
    "produto para saúde": "Produtos para Saude",
    "produtos para saúde": "Produtos para Saude",
    "produtos para saude": "Produtos para Saude",
    "produto para saude": "Produtos para Saude",
}
```

Caso a coluna `TIPO_DE_PRODUTO` não esteja presente, um aviso será exibido, mas o processo de consolidação seguirá normalmente.

---

### 📁 Organização da Saída

O arquivo final é salvo como:

```
dataframe/consolidado/consolidado_final.xlsx
```

Este arquivo contém os dados de **todos os anos**, com colunas padronizadas e valores tratados, pronto para análises exploratórias, estatísticas ou visualizações.

---

### ✅ Resultado Esperado

- Um único arquivo `.xlsx` consolidado, com todas as informações limpas, padronizadas e integradas.
- Dados provenientes tanto dos arquivos tratados quanto dos arquivos adicionais dentro de `dados/`, exceto 2023.

---

In [ ]:
import os
import pandas as pd
import warnings

# Oculta avisos do openpyxl
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# Caminhos
PASTA_TRATADO = "dataframe/tratado"
PASTA_DADOS = "dados"
PASTA_EXCLUIDA = os.path.join(PASTA_DADOS, "2023")
PASTA_OUTPUT = "dataframe/consolidado"
os.makedirs(PASTA_OUTPUT, exist_ok=True)

# Lista para armazenar os DataFrames
dataframes = []

# === 1. Ler arquivos da pasta "dataframe/tratado" ===
for arquivo in os.listdir(PASTA_TRATADO):
    if arquivo.endswith(".xlsx"):
        caminho_arquivo = os.path.join(PASTA_TRATADO, arquivo)
        try:
            df = pd.read_excel(caminho_arquivo, sheet_name=0, engine="openpyxl")
            dataframes.append(df)
        except Exception as e:
            print(f"❌ Erro ao ler {arquivo} em tratado: {e}")

# === 2. Ler arquivos .xlsx de todas as subpastas da pasta "dados", exceto "dados/2023" ===
for raiz, _, arquivos in os.walk(PASTA_DADOS):
    if os.path.commonpath([raiz, PASTA_EXCLUIDA]) == PASTA_EXCLUIDA:
        continue  # pula a pasta 'dados/2023' e suas subpastas
    for arquivo in arquivos:
        if arquivo.endswith(".xlsx"):
            caminho_arquivo = os.path.join(raiz, arquivo)
            try:
                df = pd.read_excel(caminho_arquivo, sheet_name=0, engine="openpyxl")
                dataframes.append(df)
            except Exception as e:
                print(f"❌ Erro ao ler {arquivo} em dados: {e}")

# === 3. Concatenar e remover duplicatas ===
if dataframes:
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    df_consolidado = df_consolidado.drop_duplicates()

    # === 4. Padronizar valores da coluna TIPO_DE_PRODUTO ===
    if "TIPO_DE_PRODUTO" in df_consolidado.columns:
        df_consolidado["TIPO_DE_PRODUTO"] = df_consolidado["TIPO_DE_PRODUTO"].str.lower().str.strip()

        mapeamento = {
            "alimentos": "Alimento",
            "alimento": "Alimento",
            "cosméticos": "Cosmético",
            "cosmeticos": "Cosmético",
            "medicamentos": "Medicamento",
            "produto para saúde": "Produtos para Saude",
            "produtos para saúde": "Produtos para Saude",
            "produtos para saude": "Produtos para Saude",
            "produto para saude": "Produtos para Saude",
        }

        df_consolidado["TIPO_DE_PRODUTO"] = df_consolidado["TIPO_DE_PRODUTO"].map(mapeamento).fillna(df_consolidado["TIPO_DE_PRODUTO"])
    else:
        print("⚠️ Coluna 'TIPO_DE_PRODUTO' não encontrada.")

    # Salvar resultado
    output_path = os.path.join(PASTA_OUTPUT, "consolidado_final.xlsx")
    df_consolidado.to_excel(output_path, index=False)
    print(f"✅ Consolidação final salva em: {output_path}")
else:
    print("⚠️ Nenhum arquivo .xlsx encontrado.")